In [25]:
# !pip install langchain=0.0.348
# !pip install openai=0.28

## Imports

In [26]:
import os
import openai
from langchain.chat_models import AzureChatOpenAI
from langchain.agents import Tool
from langchain.tools import Tool
from langchain.memory import ConversationBufferMemory
from dotenv import load_dotenv,find_dotenv

from langchain.agents import AgentType
from langchain.agents import initialize_agent

# Set environment variables

In [27]:
load_dotenv()

True

In [28]:
openai.api_key = os.getenv("api_key")
openai.api_base = os.getenv("api_base")
openai.api_type = os.getenv("api_type")
openai.api_version = os.getenv("api_version")
deployment_id = os.getenv("deployment_id_gpt_4")
gpt_model = os.getenv("deployment_id_gpt_4")
model_engine = os.getenv("deployment_id_gpt_4")
embd_model = 'text-embedding-ada-002'

os.environ.update({
    "OPENAI_API_TYPE": os.getenv("api_type"),
    "OPENAI_API_VERSION": os.getenv("api_version"),
    "OPENAI_API_BASE": os.getenv("api_base"),
    "OPENAI_API_KEY": os.getenv("api_key")
})

llm = AzureChatOpenAI(openai_api_base=openai.api_base, temperature=0.5,
                      deployment_name=model_engine, openai_api_version="2023-03-15-preview")

# Set your variables here

In [29]:
context_data = ""
programming_language = "python"
seed_user_input = f"I am having 10 years expirence as {programming_language} developer."
max_questions_to_be_asked = 5

## Function to play the role of first person. (interviewer)

In [30]:
def interviever_roleplay(input_resp, context_data):
    system_content = f"""Act as an {programming_language} interviewer and your job is to ask technical questions about {programming_language} based on the user \
    input and the context conversation data passed as context. \
    you need to ask one interview question at a time based on the previous context. \
    Evaluvate the answer and give feedback and ask a follow up question. \
    Appreciate the user for the efforts of answering the question and also you should point mistakes in the answer if any in a very polite manner.

    context:{context_data}
    """
    
    conversation_interviewer = [{"role": "system", "content":system_content}]

    user_input = {"role":"user", "content":input_resp}
    conversation_interviewer.append(user_input)
    response = openai.ChatCompletion.create(engine=model_engine, messages=conversation_interviewer, temperature=0.5)

    resp = response['choices'][0]['message']['content'].strip()
    # print(resp)
    return resp

In [31]:
resp = interviever_roleplay(seed_user_input, "")
print(resp)

That's impressive! You must have a deep understanding of Python. Let's start with a simple question. Can you explain the differences between a list and a tuple in Python?


## Function to play the role of second person. (candidate)

In [32]:
def candidate_roleplay(input_resp, context_data):
    system_content = f"""Act as an expirence {programming_language} developer you have to answer technical questions asked as user input. \
    you need to give a very short explaination and use example to explain if asked. \
    Consider the context of the conversation passed as context while answering the question. \

    context:{context_data}
    """
    conversation_candidate = [{"role": "system", "content":system_content}]
    user_input = {"role":"user", "content":input_resp}
    
    conversation_candidate.append(user_input)
    response = openai.ChatCompletion.create(engine=model_engine, messages=conversation_candidate, temperature=0.5)

    resp = response['choices'][0]['message']['content'].strip()
    # print(resp)
    return resp
    # return response.choices[0].message.content.strip()

In [33]:
resp = candidate_roleplay(f"What are features of {programming_language}?", "")
print(resp)

Python has several key features that make it a popular programming language:

1. Readability: Python has a clean syntax that is easy to read and write, which makes it great for beginners. The use of indentation for block delimiters is unique to Python.

2. Interpreted Language: Python is an interpreted language, meaning that it executes code line by line, which makes debugging easier.

3. Dynamically Typed: In Python, you don’t have to declare the data type of a variable. The interpreter infers the data type at runtime.

4. Large Standard Library: Python has a robust standard library that includes modules for string operations, Internet, web service tools, operating system interfaces, and protocols.

5. Cross-platform: Python is portable and can be used on any operating system including Windows, Linux, and Mac.

6. Support for Object-Oriented and Structured Programming: Python supports both object-oriented programming and structured programming.

7. Extensible: Python can be extended t

## Function to generate the summary of the conversation between interviewer and candidate

In [34]:
def interview_summry(context_data):
    system_content = f"""You are an expert in {programming_language} and you need to evaluvate a candidate based on the conversation passed as user_input here. \
    evaluvate the context and give feedback for the candidate based on his knowledge. \
    You need to identify the good aspects as well as areas need improvement for the candidate based on the context. \
    Provide your feedback in a short point wise paragraph. \
    You need to say thanks for having this discussion. \
    """
    conversation_candidate = [{"role": "system", "content":system_content}]
    user_input = {"role":"user", "content":context_data}
    
    conversation_candidate.append(user_input)
    response = openai.ChatCompletion.create(engine=model_engine, messages=conversation_candidate, temperature=0.5)

    resp = response['choices'][0]['message']['content'].strip()
    # print(resp)
    return resp
    # return response.choices[0].message.content.strip()

## Execution

In [35]:
question_asked_flag = "not_set"

resp = seed_user_input
question_count = 1

context_data = "introduction: "+ resp +"\n"

print("Introduction of the candidate : "+ resp)
print("======================================================================")

while True:
    if question_count > max_questions_to_be_asked:
        break
    
    if question_asked_flag == "not_set":
        resp = interviever_roleplay(resp, context_data)
        
        print(f"Interviewer : Question {question_count}")
        
        question_asked_flag = "True"
        context_data = context_data+"\n Interviewer: "+ resp 
        
    elif question_asked_flag == "False":
        resp = interviever_roleplay(resp, context_data)
        
        print(f"Interviewer : Question {question_count}")
        
        question_asked_flag = "True"
        context_data = context_data+"\n Candidate: "+ resp 
        
    elif question_asked_flag == "True":
        resp = candidate_roleplay(resp, context_data)
         
        print("Candidate : ")

        question_count = question_count+1
        question_asked_flag = "False"
        context_data = context_data+"\n Interviewer: "+ resp 
    

        
    # print("======================================================================")
    print(resp)
    print("======================================================================")


resp = interview_summry(context_data)
print("<<<<<<<<<<<<<<<<<<>>>>>>>>>>>>>>>>>>>>>>>>>>")
print("Conclusion : ")
print(resp)
print("<<<<<<<<<<<<<<<<<<>>>>>>>>>>>>>>>>>>>>>>>>>>")

Introduction of the candidate : I am having 10 years expirence as python developer.
Interviewer : Question 1
That's impressive! With your extensive experience, you must have a deep understanding of Python. Let's start with a basic question: Can you explain the difference between a list and a tuple in Python?
Candidate : 
Sure, a list and a tuple are quite similar as they are both sequence types that can store a collection of items. The primary difference between them is that lists are mutable while tuples are immutable.

This means that we can change, add, or remove items after the list is created. Here's an example:

```python
list_example = [1, 2, 3]
list_example[0] = 0  # This will change the first item to 0
```

On the other hand, once a tuple is created, we cannot change its value. Here's an example:

```python
tuple_example = (1, 2, 3)
tuple_example[0] = 0  # This will raise a TypeError
```

So, in summary, if you need a sequence type where you might need to change the elements, 